[GDG Ahlen / Incremental design of LLM-powered agentic applications](https://www.youtube.com/watch?v=uIQlMSX5gx4)

Resources:
- [Google AI Studio](https://python.langchain.com/docs/integrations/chat/google_generative_ai/)
     - need to get `GOOGLE_API_KEY`
- [LangChain: GoogleGenerativeAI](https://python.langchain.com/docs/integrations/chat/google_generative_ai/)


# Module 1: Basics of instrumenting LLM with Gemini

## Setup Gemini in LangChain

Important to set Experimental model of Gemini in `ChatGoogleGenerativeAI.model` 

In [ ]:
# Import env variables from `.env` file
from dotenv import load_dotenv
load_dotenv()

# Check for Google GEMINI API KEY
import os
assert "GOOGLE_API_KEY" in os.environ

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    max_tokens=None,
    model="gemini-2.0-flash-exp",
    temperature=1,
    top_k=1,
    top_p=0.9
)


## Simple prompt

In [10]:
prompt = ChatPromptTemplate.from_template("Process this text: {input}")

chain = prompt | llm | StrOutputParser()

result = chain.invoke({
    "input": "Tell me who are you, dear?"
})
print(result)


The text is a question: "Tell me who are you, dear?"

It indicates:

*   **A request for identification:** Someone is asking another to identify themselves.
*   **Informality:** The use of "dear" suggests a familiar or affectionate tone.


## System prompt

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are export in {topic} and should answering to user inputs, provide concise and simple answer, also suggest tradeoffs",
        ),
        (
            "user", 
            "{input}"
        )
    ]
)

chain = prompt | llm | StrOutputParser()
chain.invoke(
    {
        "topic": "Maching Learning",
        "input": "I love programming.",
    }
)

"That's great! Programming is a valuable and versatile skill. Do you have any specific areas of programming you're interested in exploring, like web development, data science, or game development?  I can help you find resources or understand different paths."

## Multi-modal prompt with structured response

### Given image

![llm_in_production.png](images/llm_in_production.png)

### Multimodal prompt without structured response

In [ ]:

import base64
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

with open("images/llm_in_production.png", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

response = llm.invoke([
    HumanMessage(content=[
        {"type": "text", "text": "What's on this image?"},
        {"type": "image_url", "image_url": f"data:image/jpeg;base64,{encoded_image}"}
    ])
])

print(response)

content='The image is a diagram illustrating an LLM (Large Language Model) pipeline and its applications.\n\nHere\'s a breakdown:\n\n**Top Level: AI Application + Data Products**\n\n*   **Q&A Webapp:** A web application that allows users to ask questions and receive answers generated by an LLM.\n*   **Chatbot:** A conversational AI system that interacts with users through text or voice.\n*   **Model as an API:** The LLM is exposed as an API (Application Programming Interface), allowing other applications to access and use its capabilities.\n\n**Middle Level: LLM Pipeline**\n\nThis section describes the steps involved in processing text using an LLM.\n\n*   **Corpus Creation:** Gathering and preparing a large dataset of text for training the LLM.\n*   **Text Pre-processing:** Cleaning and formatting the text data to make it suitable for the LLM.\n*   **Prompt Engineering:** Crafting specific prompts or instructions to guide the LLM\'s responses.\n*   **LLM Inference:** Using the LLM to 

### Multimodal prompt with structured response

In [26]:
from pydantic import BaseModel, Field
from typing import List
from rich import print

class LLModel(BaseModel):
    model: str = Field(description="Model name")
    company: str = Field(description="The company name created model")
    parameters: str = Field(description="Rough estimation of number of model parameters")
    pros: str = Field(description="Pros of the model")
    cons: str = Field(description="Cons of the model")

class ResponseSchema(BaseModel):
    models: List[LLModel]


llm_structured = llm.with_structured_output(ResponseSchema)


with open("images/llm_in_production.png", "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

response = llm_structured.invoke([
    HumanMessage(content=[
        {"type": "text", "text": "Provide information about LLM models on the image"},
        {"type": "image_url", "image_url": f"data:image/jpeg;base64,{encoded_image}"}
    ])
])

print(response)


ResponseSchema(
    models=[
        LLModel(model='GPT 3.5', company='OpenAI', parameters='175 billion', pros='Versatile', cons='Costly'),
        LLModel(
            model='GPT 4.0',
            company='OpenAI',
            parameters='1.76 trillion',
            pros='Cutting edge',
            cons='Very costly'
        ),
        LLModel(
            model='LLaMA',
            company='Meta',
            parameters='7B to 65B',
            pros='Open source',
            cons='Not as versatile as others'
        ),
        LLModel(
            model='Hugging Face Models',
            company='Hugging Face',
            parameters='Varies',
            pros='Open source and community support',
            cons='Can be difficult to implement'
        ),
        LLModel(
            model='MPT',
            company='MPT',
            parameters='7B to 30B',
            pros='Open source',
            cons='Not as versatile as others'
        )
    ]
)

## Batch prompts

### Batch prompts without structrued response

In [ ]:
prompt_template = ChatPromptTemplate.from_template("Concise explanating using less 50 words of following thing: {input}")

chain = prompt_template | llm | StrOutputParser()

prompt_values = [
    {"input": "Feedforward Neural Networks"},
    {"input": "Convolutional Neural Networks"},
    {"input": "Recurrent Neural Networks"},
    {"input": "Generative Adversarial Networks"},
]

result = chain.batch(prompt_values)

# we will get a list with the two plain text outputs
print(result)

[
    "Feedforward neural networks are the simplest type of artificial neural network. Information flows in one 
direction: from input to output. They consist of layers of interconnected nodes, where each connection has a 
weight. These weights are adjusted during training to minimize the difference between the network's output and the 
desired output.",
    'Convolutional Neural Networks (CNNs) are a type of deep learning architecture primarily used for image and 
video processing. They use convolutional layers to automatically and adaptively learn spatial hierarchies of 
features from input data. Pooling layers reduce dimensionality, and fully connected layers perform 
classification.',
    'Recurrent Neural Networks (RNNs) are designed for sequential data. They have a "memory" of past inputs, using 
feedback loops to process information over time. This makes them suitable for tasks like language modeling and time
series analysis.',
    'Generative Adversarial Networks (GANs) use two neural networks: a generator that creates fake data and a 
discriminator that tries to distinguish between real and fake data. Through adversarial training, the generator 
learns to produce increasingly realistic data, fooling the discriminator.',
    'A Transformer is a neural network architecture that uses self-attention mechanisms to weigh the importance of 
different parts of the input sequence. This allows it to handle long-range dependencies and is commonly used in 
natural language processing for tasks like translation and text generation.'
]

### Batch prompts with structrued response

In [33]:
class NeuralNetworkModel(BaseModel):
    name: str = Field(description="Name")
    year: str = Field(description="The year of creation")
    common_usage: str = Field(description="Fields where commonly applicable")
    pros: str = Field(description="Pros of the architecture")
    cons: str = Field(description="Cons of the architecture")

class ResponseSchema(BaseModel):
    neural_networks: List[NeuralNetworkModel]

prompt_template = ChatPromptTemplate.from_template("Provide concise information for: {input}")
llm_structured = llm.with_structured_output(ResponseSchema)

chain = prompt_template | llm_structured 

prompt_values = [
    {"input": "Feedforward Neural Networks"},
    {"input": "Convolutional Neural Networks"},
    {"input": "Recurrent Neural Networks"},
    {"input": "Generative Adversarial Networks"}
]

result = chain.batch(prompt_values)

# we will get a list with the two plain text outputs
print(result)

[
    ResponseSchema(
        neural_networks=[
            NeuralNetworkModel(
                name='Feedforward Neural Networks',
                year='1958',
                common_usage='Image recognition, natural language processing',
                pros='Simple to implement, well-suited for many tasks',
                cons='Can be computationally expensive, prone to overfitting'
            )
        ]
    ),
    ResponseSchema(
        neural_networks=[
            NeuralNetworkModel(
                name='Convolutional Neural Networks',
                year='1989',
                common_usage='image recognition, object detection',
                pros='Excellent feature extraction, translation invariance',
                cons='computationally expensive, requires large datasets'
            )
        ]
    ),
    ResponseSchema(
        neural_networks=[
            NeuralNetworkModel(
                name='Recurrent Neural Networks',
                year='1980',
                common_usage='Good for processing sequential data',
                pros='Can handle variable length inputs',
                cons='Vanishing gradients problem'
            )
        ]
    ),
    ResponseSchema(
        neural_networks=[
            NeuralNetworkModel(
                name='Generative Adversarial Networks',
                year='2014',
                common_usage='image generation, data augmentation, and anomaly detection',
                pros='capable of generating high-quality and realistic samples, and can learn complex data 
distributions',
                cons='training instability, mode collapse, and difficulty in evaluating the quality of generated 
samples'
            )
        ]
    )
]